In [1]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import sqlite3

# ML utilities
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



In [5]:
# Optional libraries
from xgboost import XGBClassifier
import lightgbm as lgb
from imblearn.over_sampling import SMOTE


In [7]:
DATA_PATH = Path("creditcard.csv")
DB_PATH = Path("fraud_detection.db")

RANDOM_STATE = 42
TEST_SIZE = 0.30
N_SPLITS_CV = 5
BATCH_SIZE = 5000

FALSE_POSITIVE_COST_RATE = 0.005

THRESHOLDS = np.linspace(0.1, 0.9, 9)

np.random.seed(RANDOM_STATE)


In [9]:
df = pd.read_csv(DATA_PATH)

# Create Transaction_ID 
if "Transaction_ID" not in df.columns:
    df.insert(0, "Transaction_ID", np.arange(1, len(df) + 1))

assert "Class" in df.columns, "Dataset must contain 'Class' column."
assert df["Class"].isin([0, 1]).all(), "Class column must be 0/1."

fraud_rate = df["Class"].mean()
print(f"Fraud rate: {fraud_rate:.4%}")


Fraud rate: 0.1727%


In [11]:
#Features
feature_cols = [c for c in df.columns if c not in ["Transaction_ID", "Class"]]

X = df[feature_cols].copy()
y = df["Class"].copy()
ids = df["Transaction_ID"].copy()


In [13]:
#Train - Test Split
X_train, X_test, y_train, y_test,id_train,id_test = train_test_split(X,y,ids,test_size=TEST_SIZE,random_state=RANDOM_STATE,stratify=y)

In [15]:
print("Training Set",X_train.shape)
print("Test set", X_test.shape)

df["Dataset_Split"] = "Train"
df.loc[df["Transaction_ID"].isin(id_test), "Dataset_Split"] = "Test"

Training Set (199364, 30)
Test set (85443, 30)


In [17]:
#test metadata
test_info = (pd.DataFrame({"Transaction_ID":id_test}).merge(df[["Transaction_ID","Amount","Class"]], on ="Transaction_ID",how="left")
             .rename(columns={"Class":"Actual_Class"}))
test_info = test_info.sort_values("Transaction_ID").reset_index(drop=True)
test_info["Batch_ID"] = (np.arange(len(test_info)) // BATCH_SIZE) + 1

In [19]:
#Align X_test to test_info

X_test = pd.DataFrame(X_test, columns=feature_cols)

# Add Transaction_ID to X_test
X_test["Transaction_ID"] = id_test.values

# Align X_test to match test_info order exactly
X_test = test_info[["Transaction_ID"]].merge(
    X_test,
    on="Transaction_ID",
    how="left"
)

# Extract numpy matrix for prediction
X_test_input = X_test[feature_cols].values


In [21]:
#Standardize Features 
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[feature_cols])
X_test_scaled = scaler.transform(X_test[feature_cols])

y_test = test_info["Actual_Class"].values

In [34]:

try:
    import xgboost
    HAS_XGB = True
except ImportError:
    HAS_XGB = False

try:
    import lightgbm
    HAS_LGBM = True
except ImportError:
    HAS_LGBM = False
try:
    from imblearn.over_sampling import SMOTE
    HAS_SMOTE = True
except ImportError:
    HAS_SMOTE = False


In [28]:
#Machine Learning Models 
MODEL_CONFIGS = {
    "log_reg": {
        "friendly_name": "Logistic Regression",
        "use_scaled": True,
        "estimator": LogisticRegression(
            class_weight="balanced",
            max_iter=800,
            solver="lbfgs",
            random_state=RANDOM_STATE
        )
    },

    "rf": {
        "friendly_name": "Random Forest",
        "use_scaled": False,
        "estimator": RandomForestClassifier(
            n_estimators=400,
            class_weight="balanced",
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
    },

    "gb": {
        "friendly_name": "Gradient Boosting",
        "use_scaled": False,
        "estimator": GradientBoostingClassifier(
            random_state=RANDOM_STATE
        )
    }
}


if HAS_XGB:
    MODEL_CONFIGS["xgb"] = {
        "friendly_name": "XGBoost",
        "use_scaled": False,
        "estimator": XGBClassifier(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            eval_metric="logloss",
            tree_method="hist",
            scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
            random_state=RANDOM_STATE
        )
    }


if HAS_LGBM:
    MODEL_CONFIGS["lgbm"] = {
        "friendly_name": "LightGBM",
        "use_scaled": False,
        "estimator": lgb.LGBMClassifier(
            n_estimators=400,
            learning_rate=0.05,
            class_weight="balanced",
            random_state=RANDOM_STATE
        )
    }

print("Models loaded:")
for k, v in MODEL_CONFIGS.items():
    print(f"- {v['friendly_name']} ({k})")


Models loaded:
- Logistic Regression (log_reg)
- Random Forest (rf)
- Gradient Boosting (gb)
- XGBoost (xgb)
- LightGBM (lgbm)


In [30]:
#Cross Validation Setup 
# Define the cross-validation strategy
cv = StratifiedKFold(
    n_splits=N_SPLITS_CV,
    shuffle=True,
    random_state=RANDOM_STATE
)

# Define which scoring metrics to compute during cross-validation
cv_scoring = {
    "precision": "precision",
    "recall": "recall",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

print("Cross-validation setup complete.")


Cross-validation setup complete.


In [36]:
#SMOTE Oversampling 
if HAS_SMOTE:
    print("SMOTE detected — applying oversampling to training data.")
    smote = SMOTE(random_state=RANDOM_STATE)
    X_train_sm, y_train_sm = smote.fit_resample(X_train[feature_cols], y_train)
    X_train_scaled_sm = scaler.fit_transform(X_train_sm)
else:
    print("SMOTE not available , using original imbalanced training data.")
    X_train_sm = X_train[feature_cols]
    y_train_sm = y_train
    X_train_scaled_sm = X_train_scaled


SMOTE detected — applying oversampling to training data.


In [38]:

# Cross-validate all models

cv_results = []

for key, cfg in MODEL_CONFIGS.items():
    print(f"\nRunning cross-validation for: {cfg['friendly_name']}")

    
    if cfg["use_scaled"]:
        X_cv = X_train_scaled
    else:
        X_cv = X_train[feature_cols].values

    est = cfg["estimator"]

    scores = cross_validate(
        est,
        X_cv,
        y_train,
        cv=cv,
        scoring=cv_scoring,
        n_jobs=-1,
        return_train_score=False
    )

    result_row = {
        "model_key": key,
        "model_name": cfg["friendly_name"],
        "cv_precision_mean": scores["test_precision"].mean(),
        "cv_recall_mean": scores["test_recall"].mean(),
        "cv_f1_mean": scores["test_f1"].mean(),
        "cv_roc_auc_mean": scores["test_roc_auc"].mean()
    }
    cv_results.append(result_row)

model_cv_results = pd.DataFrame(cv_results)
model_cv_results = model_cv_results.sort_values("cv_roc_auc_mean", ascending=False).reset_index(drop=True)

print("\nCross-validation summary:")
display(model_cv_results)



Running cross-validation for: Logistic Regression

Running cross-validation for: Random Forest

Running cross-validation for: Gradient Boosting

Running cross-validation for: XGBoost

Running cross-validation for: LightGBM

Cross-validation summary:


,model_key,model_name,cv_precision_mean,cv_recall_mean,cv_f1_mean,cv_roc_auc_mean
0,log_reg,Logistic Regression,0.070008,0.918542,0.130055,0.982181
1,xgb,XGBoost,0.887134,0.831330,0.857720,0.981497
2,lgbm,LightGBM,0.909430,0.813939,0.858924,0.975493
3,rf,Random Forest,0.947677,0.770332,0.849311,0.956614
4,gb,Gradient Boosting,0.788523,0.519608,0.605154,0.672079


In [40]:
# Pick the model with the best ROC AUC 
champion_key = model_cv_results.iloc[0]["model_key"]
champion_name = model_cv_results.iloc[0]["model_name"]

print(f"\nSelected champion model: {champion_name} ({champion_key})")

champion_cfg = MODEL_CONFIGS[champion_key]
champion_est = champion_cfg["estimator"]
use_scaled = champion_cfg["use_scaled"]



Selected champion model: Logistic Regression (log_reg)


In [42]:
#Train the best model 
if HAS_SMOTE:
    print("\nTraining champion model on SMOTE-oversampled data.")
    if use_scaled:
        X_train_final = X_train_scaled_sm
    else:
        X_train_final = X_train_sm.values
    y_train_final = y_train_sm
else:
    print("\nTraining champion model on original imbalanced data.")
    if use_scaled:
        X_train_final = X_train_scaled
    else:
        X_train_final = X_train[feature_cols].values
    y_train_final = y_train



Training champion model on SMOTE-oversampled data.


In [44]:
#Print the champion model 
champion_est.fit(X_train_final,y_train_final)

LogisticRegression(class_weight='balanced', max_iter=800, random_state=42)

In [46]:
#Prepare test input 
if use_scaled:
    X_test_input = X_test_scaled
else:
    X_test_input = X_test[feature_cols].values

# Predict probabilities for positive class (fraud)
y_proba_test = champion_est.predict_proba(X_test_input)[:, 1]

# Build prediction table for the test set
predictions_df = test_info.copy()
predictions_df["Model_Key"] = champion_key
predictions_df["Model_Name"] = champion_name
predictions_df["Pred_Prob"] = y_proba_test

display(predictions_df.head())

,Transaction_ID,Amount,Actual_Class,Batch_ID,Model_Key,Model_Name,Pred_Prob
0,3,378.66,0,1,log_reg,Logistic Regression,1.0
1,6,3.67,0,1,log_reg,Logistic Regression,1.0
2,8,40.80,0,1,log_reg,Logistic Regression,1.0
3,10,3.68,0,1,log_reg,Logistic Regression,1.0
4,11,7.80,0,1,log_reg,Logistic Regression,1.0


In [48]:

#Threshold tuning and global financial metrics


def evaluate_threshold(y_true, y_score, amounts, threshold, fp_cost_rate):
    """
    Helper function to compute classification + financial metrics
    for a given probability threshold.
    """
    y_pred = (y_score >= threshold).astype(int)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_true, y_score)

    is_fraud = (y_true == 1)
    flagged = (y_pred == 1)

    tp_mask = is_fraud & flagged
    fp_mask = (~is_fraud) & flagged
    fn_mask = is_fraud & (~flagged)

    fraud_amount_total = amounts[is_fraud].sum()
    prevented_fraud_amount = amounts[tp_mask].sum()
    missed_fraud_amount = amounts[fn_mask].sum()
    false_positive_cost = (amounts[fp_mask] * fp_cost_rate).sum()
    net_savings = prevented_fraud_amount - false_positive_cost

    return {
        "threshold": threshold,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "tp": tp,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "fraud_amount_total": fraud_amount_total,
        "prevented_fraud_amount": prevented_fraud_amount,
        "missed_fraud_amount": missed_fraud_amount,
        "false_positive_cost": false_positive_cost,
        "net_savings": net_savings
    }

threshold_results = []

amounts_test = predictions_df["Amount"].values
y_true_test = predictions_df["Actual_Class"].values

for thr in THRESHOLDS:
    metrics = evaluate_threshold(
        y_true=y_true_test,
        y_score=y_proba_test,
        amounts=amounts_test,
        threshold=thr,
        fp_cost_rate=FALSE_POSITIVE_COST_RATE
    )
    threshold_results.append(metrics)

threshold_metrics_df = pd.DataFrame(threshold_results)
threshold_metrics_df = threshold_metrics_df.sort_values("threshold").reset_index(drop=True)

print("\nThreshold tuning results:")
display(threshold_metrics_df)



Threshold tuning results:


,threshold,tn,fp,fn,tp,precision,recall,f1,roc_auc,fraud_amount_total,prevented_fraud_amount,missed_fraud_amount,false_positive_cost,net_savings
0,0.1,2705,82590,0,148,0.001789,1.0,0.003571,0.935049,19311.89,19311.89,0.0,36415.31455,-17103.42455
1,0.2,2825,82470,0,148,0.001791,1.0,0.003576,0.935049,19311.89,19311.89,0.0,36347.51915,-17035.62915
2,0.3,2892,82403,0,148,0.001793,1.0,0.003579,0.935049,19311.89,19311.89,0.0,36316.90770,-17005.01770
3,0.4,2965,82330,0,148,0.001794,1.0,0.003582,0.935049,19311.89,19311.89,0.0,36296.37930,-16984.48930
4,0.5,3030,82265,0,148,0.001796,1.0,0.003585,0.935049,19311.89,19311.89,0.0,36234.97555,-16923.08555
5,0.6,3088,82207,0,148,0.001797,1.0,0.003588,0.935049,19311.89,19311.89,0.0,36210.92185,-16899.03185
6,0.7,3161,82134,0,148,0.001799,1.0,0.003591,0.935049,19311.89,19311.89,0.0,36177.76125,-16865.87125
7,0.8,3236,82059,0,148,0.001800,1.0,0.003594,0.935049,19311.89,19311.89,0.0,36096.89810,-16785.00810
8,0.9,3371,81924,0,148,0.001803,1.0,0.003600,0.935049,19311.89,19311.89,0.0,36021.32595,-16709.43595


In [50]:
# Pick best threshold by F1
best_f1_idx = threshold_metrics_df["f1"].idxmax()
best_threshold = threshold_metrics_df.loc[best_f1_idx, "threshold"]
best_f1 = threshold_metrics_df.loc[best_f1_idx, "f1"]
best_net_savings = threshold_metrics_df.loc[best_f1_idx, "net_savings"]

print(f"\nBest threshold by F1: {best_threshold:.2f}")
print(f"F1 at best threshold: {best_f1:.4f}")
print(f"Net savings at best threshold: {best_net_savings:,.2f}")



Best threshold by F1: 0.90
F1 at best threshold: 0.0036
Net savings at best threshold: -16,709.44


In [52]:

# Apply best threshold and compute batch-level financials


# Final predicted labels using best threshold
predictions_df["Pred_Class"] = (predictions_df["Pred_Prob"] >= best_threshold).astype(int)

# Mark classification outcomes
predictions_df["Is_Fraud"] = (predictions_df["Actual_Class"] == 1)
predictions_df["Is_Flagged"] = (predictions_df["Pred_Class"] == 1)

predictions_df["Is_TP"] = predictions_df["Is_Fraud"] & predictions_df["Is_Flagged"]
predictions_df["Is_FP"] = ~predictions_df["Is_Fraud"] & predictions_df["Is_Flagged"]
predictions_df["Is_FN"] = predictions_df["Is_Fraud"] & ~predictions_df["Is_Flagged"]
predictions_df["Is_TN"] = ~predictions_df["Is_Fraud"] & ~predictions_df["Is_Flagged"]

# Financial columns
predictions_df["Fraud_Amount"] = np.where(predictions_df["Is_Fraud"], predictions_df["Amount"], 0.0)
predictions_df["Prevented_Fraud_Amount"] = np.where(predictions_df["Is_TP"], predictions_df["Amount"], 0.0)
predictions_df["Missed_Fraud_Amount"] = np.where(predictions_df["Is_FN"], predictions_df["Amount"], 0.0)
predictions_df["False_Positive_Cost"] = np.where(
    predictions_df["Is_FP"],
    predictions_df["Amount"] * FALSE_POSITIVE_COST_RATE,
    0.0
)
predictions_df["Net_Savings"] = predictions_df["Prevented_Fraud_Amount"] - predictions_df["False_Positive_Cost"]

# Add the chosen threshold as a column for Power BI
predictions_df["Chosen_Threshold"] = best_threshold

display(predictions_df.head())


,Transaction_ID,Amount,Actual_Class,Batch_ID,Model_Key,Model_Name,Pred_Prob,Pred_Class,Is_Fraud,Is_Flagged,Is_TP,Is_FP,Is_FN,Is_TN,Fraud_Amount,Prevented_Fraud_Amount,Missed_Fraud_Amount,False_Positive_Cost,Net_Savings,Chosen_Threshold
0,3,378.66,0,1,log_reg,Logistic Regression,1.0,1,False,True,False,True,False,False,0.0,0.0,0.0,1.89330,-1.89330,0.9
1,6,3.67,0,1,log_reg,Logistic Regression,1.0,1,False,True,False,True,False,False,0.0,0.0,0.0,0.01835,-0.01835,0.9
2,8,40.80,0,1,log_reg,Logistic Regression,1.0,1,False,True,False,True,False,False,0.0,0.0,0.0,0.20400,-0.20400,0.9
3,10,3.68,0,1,log_reg,Logistic Regression,1.0,1,False,True,False,True,False,False,0.0,0.0,0.0,0.01840,-0.01840,0.9
4,11,7.80,0,1,log_reg,Logistic Regression,1.0,1,False,True,False,True,False,False,0.0,0.0,0.0,0.03900,-0.03900,0.9


In [54]:
batch_financials = (
    predictions_df
    .groupby("Batch_ID")
    .agg(
        Transactions=("Transaction_ID", "count"),
        Fraud_Cases=("Is_Fraud", "sum"),
        TP=("Is_TP", "sum"),
        FP=("Is_FP", "sum"),
        FN=("Is_FN", "sum"),
        TN=("Is_TN", "sum"),
        Total_Amount=("Amount", "sum"),
        Fraud_Amount=("Fraud_Amount", "sum"),
        Prevented_Fraud_Amount=("Prevented_Fraud_Amount", "sum"),
        Missed_Fraud_Amount=("Missed_Fraud_Amount", "sum"),
        False_Positive_Cost=("False_Positive_Cost", "sum"),
        Net_Savings=("Net_Savings", "sum")
    )
    .reset_index()
)

print("\nBatch-level financial summary:")
display(batch_financials.head())



Batch-level financial summary:


,Batch_ID,Transactions,Fraud_Cases,TP,FP,FN,TN,Total_Amount,Fraud_Amount,Prevented_Fraud_Amount,Missed_Fraud_Amount,False_Positive_Cost,Net_Savings
0,1,5000,26,26,4795,0,179,343039.77,5011.00,5011.00,0.0,1655.69505,3355.30495
1,2,5000,6,6,4862,0,132,494609.52,451.73,451.73,0.0,2390.85920,-1939.12920
2,3,5000,17,17,4798,0,185,582561.59,2320.42,2320.42,0.0,2702.72785,-382.30785
3,4,5000,4,4,4826,0,170,502522.80,102.75,102.75,0.0,2468.27045,-2365.52045
4,5,5000,9,9,4789,0,202,539608.92,1162.10,1162.10,0.0,2548.59955,-1386.49955


In [56]:

# Export to SQLite for Power BI


conn = sqlite3.connect(DB_PATH)

model_cv_results.to_sql("model_cv_results", conn, if_exists="replace", index=False)
threshold_metrics_df.to_sql("threshold_metrics", conn, if_exists="replace", index=False)
predictions_df.to_sql("test_predictions", conn, if_exists="replace", index=False)
batch_financials.to_sql("batch_financials", conn, if_exists="replace", index=False)

conn.close()

print(f"\nAll tables exported to SQLite database: {DB_PATH}")
print("Tables:")
print("- model_cv_results")
print("- threshold_metrics")
print("- test_predictions")
print("- batch_financials")



All tables exported to SQLite database: fraud_detection.db
Tables:
- model_cv_results
- threshold_metrics
- test_predictions
- batch_financials


In [1]:
import os

for root, dirs, files in os.walk(".", topdown=True):
    for name in files:
        if name.endswith(".db"):
            print(os.path.join(root, name))


./fraud_detection.db
./fraud_analytics (1).db
./fraud_analytics.db


In [3]:
import sqlite3

files = ["fraud_detection.db", "fraud_analytics.db", "fraud_analytics (1).db"]

for f in files:
    print("Checking:", f)
    conn = sqlite3.connect(f)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    print(cursor.fetchall())
    conn.close()
    print("--------------")


Checking: fraud_detection.db
[('model_cv_results',), ('threshold_metrics',), ('test_predictions',), ('batch_financials',)]
--------------
Checking: fraud_analytics.db
[('transactions',), ('model_predictions',), ('model_metrics',), ('financial_metrics',)]
--------------
Checking: fraud_analytics (1).db
[('transactions',), ('model_metrics',)]
--------------
